# Metrowest machine learning self-driving car
```
The project is a self-driving car.
This notebook trains the the CNN.
Refer to the .pdf for details
  
Much of this is taken from Kiras CIFAR-10 example - see reference.

References:
https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
https://www.meetup.com/Natick-Artificial-Intelligence-Meetup/
https://github.com/experiencor.self-driving-toy-car/
http://vision.stanford.edu/teaching/cs231n/
https://www.tensorflow.org/
```

In [ ]:
# force libraries to re-load every time - helpful during development
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../lib')

# Data Loading

In [ ]:
from metrowestcar_dataset import Dataset
dataset = Dataset.data()


In [43]:

num_classes = 3

# input image dimensions
img_rows, img_cols = 64, 64
input_shape = (1, img_rows, img_cols)

# the data, split between train and test sets
dataset = Dataset.data()
x_train_raw = dataset['images_train']
x_test_raw  = dataset['images_test']
y_train_raw = dataset['steering_train']
y_test_raw  = dataset['steering_test']

# dataset is uint8 but Kiras expects float - do this conversion
# also - convert class vectors to binary (1-hot) encoded array

x_train = x_train_raw.astype('float32')
x_test = x_test_raw.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train_raw, num_classes)
y_test = keras.utils.to_categorical(y_test_raw, num_classes)

print("x_train before:" + str(type(x_train_raw[0][0][0][0])))
print("x_train after: " + str(type(x_train[0][0][0][0])))
print("y_train before " + str(type(y_train_raw)) + " " + str(y_train_raw.shape) + " " +  str(type(y_train_raw[0])))
print("y_train after  " + str(type(y_train)) + " " + str(y_train.shape) + str(type(y_train[0][0])))

x_train before:<class 'numpy.uint8'>
x_train after: <class 'numpy.float32'>
y_train before <class 'numpy.ndarray'> (54,) <class 'numpy.uint8'>
y_train after  <class 'numpy.ndarray'> (54, 3)<class 'numpy.float32'>


## Construct the Model

In [44]:
'''Trains a CNN on the dataset.
'''

from __future__ import print_function

import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
activation_19 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 62, 62, 32)        9248      
_________________________________________________________________
activation_20 (Activation)   (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 31, 31, 64)        18496     
__________

# Train the model

In [45]:
batch_size = 32
epochs = 3
data_augmentation = False

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'metrowest_keras_trained_model.h5'


# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

assert data_augmentation is False, "data augmentation is not coded"

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 54 samples, validate on 14 samples
Epoch 1/3
54/54 [==============================] - 6s 105ms/step - loss: 0.7791 - acc: 0.9074 - val_loss: 0.3015 - val_acc: 1.0000
Epoch 2/3
54/54 [==============================] - 4s 74ms/step - loss: 0.0991 - acc: 1.0000 - val_loss: 0.1412 - val_acc: 1.0000
Epoch 3/3
54/54 [==============================] - 4s 82ms/step - loss: 0.0334 - acc: 1.0000 - val_loss: 0.0873 - val_acc: 1.0000
Saved trained model at /home/chris/Documents/projects_scikit_tensorflow/test2/metrowest_scikit_tensorflow_cnn_car/model/saved_models/metrowest_keras_trained_model.h5 
14/14 [==============================] - 0s 25ms/step
Test loss: 0.08730027824640274
Test accuracy: 1.0


In [46]:
# Plot loss as a function of iteration number

In [73]:
foo = np.array([1,2,3])
print(foo.shape)
foo = foo[np.newaxis, :]
print(foo.shape)

(3,)
(1, 3)


In [84]:
# apply an image to the trained CNN
from metrowestcar_display import Displayer
displayer = Displayer()

image = x_train_raw[5]
displayer.display_image(image)
image = image.astype('float32')
image /= 255

image = image[np.newaxis, :]
steering_out = model.predict(image)
print(steering_out)

s_max = np.where(steering_out==steering_out.max())
s_index = s_max[0][0]
if s_index == 0:
    print("left")
elif s_index == 1:
    print("center")
elif s_index == 2:
    print("right")
else:
    assert False, "something wrong in prediction %r" % s_index

    

[[0.91775024 0.03758329 0.04466651]]
left
